In [1]:
# hide
%load_ext autoreload
%autoreload 2

In [2]:
# hide
from nbdev import *

In [3]:
# hide
import os
import shutil
import numpy as np
import pandas as pd
import ipywidgets as widgets
from pathlib import Path
from IPython.display import JSON

from ipyannotator.annotator import Annotator
from ipyannotator.base import Settings
from ipyannotator.mltypes import InputImage, OutputImageBbox, OutputImageLabel, NoOutput
from ipyannotator.datasets.factory_legacy import DS, get_settings, _combine_train_test
from ipyannotator.datasets.generators import create_object_detection, xyxy_to_xywh, xywh_to_xyxy
from ipyannotator.helpers import Tutorial
from ipyannotator.storage import construct_annotation_path

# Bounding Box Annotator - Identifying objects in images through boxes 

The current tutorial will illustrate how to use Ipyannotator to annotate images using bounding boxes.

The task of identifying what an image represents is called image classification. Often an image contains multiple objects which can be identified individually.

**Ipyannotator** allows users to **explore** an entire set of images and given labels; manually **create** datasets by associating labels to bounding boxes drawn on top of the images; **improve** existing annotations.

This tutorial is divided in the following steps:

- [Select dataset](#Select-Dataset)
- [Setup annotator](#Setup-annotator)
- [Explore](#Explore)
- [Create](#Create)
- [Improve](#Improve)

## Select dataset

For this tutorial an artificial minimal dataset is generated by Ipyannotator with 50 images in 2 classes to be labeled (circle and rectangle).

In [4]:
dataset = DS.ARTIFICIAL_DETECTION

## Setup annotator

This section will set up the paths and the input/output pair needed to classify the images.

The following cell imports the project file and directory where the images were generated. For this tutorial we simplify the process using the `get_settings` function instead of hardcoding the paths.

In [5]:
settings_ = get_settings(dataset)
settings_.project_file, settings_.image_dir

(Path('data/artificial_detection/annotations.json'), 'images')

Ipyannotator uses pairs of input/output data to set up the annotation. 

The Bounding Box annotator uses `InputImage` and `OutputImageBox`as the pair to set up the annotator.

The `InputImage` function provides information about the directory that contains the images to be labeled, and the images itself. The `OutputImageBox` function provides information about the directory that contains the classes that can be associated with the bounding boxes drawn on the images.

In [6]:
input_ = InputImage(image_dir=settings_.image_dir,
                    image_width=settings_.im_width,
                    image_height=settings_.im_height)

output_ = OutputImageBbox(classes=['Circle', 'Rectangle'])

input_.dir

'images'

The final part in setting up the Ipyannotator is the configuration of the `Annotator` factory with the pair of input/output data. 

The factory allows three types of annotator tools: explore, create, improve. The next sections will guide you through every step.

In [7]:
anni = Annotator(input_, output_, settings_)

## Explore

The **explore** option allows users to navigate across the images in the dataset using `next/previous` buttons. In case the dataset was already labeled, the labeling results can also be displayed. This function is used for data visualization only, improvement and addition of labels is done in the next steps. 

In [8]:
explorer = anni.explore()
explorer

BBoxAnnotatorGUI(children=(HBox(children=(Navi(children=(Button(description='< Previous', layout=Layout(width=…

Sometimes the classes are not defined yet or incomplete. To explore the input images without worring about any classes you can use the `NoOutput` option on the annotator factory which is done in the following:

In [9]:
unlabel_factory = Annotator(input_, NoOutput(), settings_)
unlabel_factory.explore()

ExploreAnnotatorGUI(children=(HBox(children=(Navi(children=(Button(description='< Previous', layout=Layout(wid…

## Create
The **create** option allows users to manually create their annotated datasets. 

The bbox annotator allows users to create multiple bounding boxes on the image and associate labels to the bboxes. Additionally features are the following:

- The lamp button can be used to highlight the annotation 
- The coordinate inputs can be changed to improve the annotated bounding box
- The trash button can delete the annotation

The next cell removes already created annotation files to create a new dataset.

In [10]:
if dataset == DS.ARTIFICIAL_DETECTION:
    dirpath = 'data/artificial_detection/create_results'
    if os.path.exists(dirpath) and os.path.isdir(dirpath):
        shutil.rmtree(dirpath)

The next cell initializes the **create** option. 

For this tutorial, a function was defined that imitates human work. You can choose between performing the annotation manually yourself or letting the function do the work for you. Use the mouse to draw on the canvas.

In [11]:
creator = anni.create()
creator

BBoxAnnotatorGUI(children=(HBox(children=(Navi(children=(Button(description='< Previous', layout=Layout(width=…

The next cell imitates human work by randomly annotating all images automatically. If you want to manually annotate then skip the next step.

In [12]:
HELPER = Tutorial(dataset, settings_.project_path)
HELPER.add_random_bboxes(creator)

0it [00:00, ?it/s]

## Improve

The **improve** feature allows users to refine the annotated dataset. 

As before, for the purpose of the tutorial, a function can be used to performe the annotation and you don't have to annotate manually. If you want to annotate manually then make sure selecting all __missed__ or __incorrect__ bounding boxes.

If you chose to annotate manually don't forget to click the __SAVE__ button when finished with each class.

In [13]:
if dataset == DS.ARTIFICIAL_DETECTION:
    dirpath = 'data/artificial_detection/create_results/missed'
    if os.path.exists(dirpath) and os.path.isdir(dirpath):
        shutil.rmtree(dirpath)

In [14]:
improver = anni.improve()

  0%|          | 0/50 [00:00<?, ?it/s]

In [15]:
improver

CaptureAnnotatorGUI(children=(HBox(children=(Navi(children=(Button(description='< Previous', layout=Layout(wid…

The next cell imitates the human work. If you chose to annotate manually make sure to skip this cell.

In [16]:
HELPER.fix_incorrect_bboxes(improver, creator)

  0%|          | 0/6 [00:00<?, ?it/s]

Now we obtain a list of all images with incorrect bboxes:

In [17]:
[k for k, v in improver.to_dict().items() if v == {'answer': True}]

['data/artificial_detection/captured/img_12.jpg',
 'data/artificial_detection/captured/img_16.jpg',
 'data/artificial_detection/captured/img_33.jpg',
 'data/artificial_detection/captured/img_35.jpg',
 'data/artificial_detection/captured/img_38.jpg',
 'data/artificial_detection/captured/img_40.jpg',
 'data/artificial_detection/captured/img_45.jpg',
 'data/artificial_detection/captured/img_46.jpg',
 'data/artificial_detection/captured/img_5.jpg',
 'data/artificial_detection/captured/img_8.jpg']